# High-performance computing 2023/24

- Name: Sascha Petznick (XXXXXX)
- Mail: S.C.Petznick@student.tudelft.nl
- Date of Submission: 21.12.2023

## Overview

- PingPong
- MM-Product
- Poisson something
- Something else

• A final report on the lab exercises must be submitted which contains all
your answers to the questions in the three exercises.
• Any figures, analysis, and tools, illustrating or demonstrating your
answers,will be beneficial for your overall grades.
• The final report should be completed independently. It should have the
following information on the front cover: your name(s), studentnumber(s),
email address(s), and submission date.
• __Submit your report to Lab reports in Assignments on Brightspace, please also
print your reports (no tedious source code) and hand in it to TA T. Deng. After
the submission please send us an email. This is to make sure that your report
is not unexpectedly got lost.__
• Submission Deadline: February 1st 2023

In [1]:
import matplotlib as pyplot
import numpy as np

# consider using https://github.com/kirbs-/hide_code to hide specific cells or 
# use second python script for creating plots

## PingPong

Messages of sizes $2^n$ where $n=0,\dots,20$ were sent between two MPI nodes. The time needed to send the message to node two, receive it there, send it back to node one, and receive it there was measured. Every message size was sent five times and the time was averaged. Moreover, the test were conducted on the supercomputer using one computing node with two processes and two computing nodes in order to analyze the difference.

The results are displayed in plots in Figure ([1](#communication-time-over-message-size)) and ([2](#communication-time-over-message-size)). Based on visual comparison it is clear that using two node has a minor effect on the experiments. Thus, the following analysis will be based on the experiment with one node.

Assuming that the message size $t_{\text{comm}}(m) = \alpha + \beta \cdot m$ is in seconds, we approximate the parameters using a linear regression as $(\alpha, \beta) = (6.49\cdot 10^{-6} [s], 5.713\cdot 10^{-9} [s/\text{bytes}])$ as opposed to the exercise sheet. The linear regression was done by dropping the first measurement in order to reduce the effect of the start-up process of MPI. The regression is displayed on Figure (1). It should be mentioned that the plots are displayed in double-logarithmis scaling and the author decided against using two linear regressions and instead used a single one as the logarithmic scaling helps identifying the overall trend and accuracy of the regression.

#### Communication time over message size

![FigureOfCommTimeOverMessageSizeN1](./figs/pingpong-doublelog_n1.svg)

![FigureOfCommTimeOverMessageSizeN2](./figs/pingpong-doublelog_n2.svg)



Based on the visualization, it is clear that the experiment for small message sizes suffers from a higher variance. This may be caused by the overall initialisation overhead that is assumed to play minor role when the message size is increased. Doing so, yields a clear trend of the function towards a linear dependence. One may argue that increasing the message size above magnitudes of $10^4$ bytes requires a second trending. Overall, this may come down to style. A possible reason for the visible jump in times when increasing the message size above $16.384$ bytes is the general size of the buffer of the MPI communicator. It might well be that messages of this size require several messages to be passed instead of fitting into a single buffer.

## MM-Product

Let $A, B \in \mathbb{R}^{n\times n}$.  

### Main idea of distributing the computations

The distribution of the matrices is as follows. We distribute the matrix $A$ to all processes using the broadcast operation of MPI. In addition, we scatter rows of $B$ in a blocksize of $\lfloor n/p\rfloor$ where $p$ is the number of processes. If it so happens that the rows is not divisible by the number of processes, the last process will work on the rest of the rows that remain when using the blocksize for all the other processes.
The following code snippet gives a general idea of the communication that is used ignoring setup and finalisation of the MPI process.

```[C]
MPI_Bcast(a, n * n, MPI_DOUBLE, 0, MPI_COMM_WORLD);
MPI_Scatterv(b, counts, displs, MPI_DOUBLE, b_local, counts[rank],
    MPI_DOUBLE, 0, MPI_COMM_WORLD);


MPI_Gatherv(c_local, counts[rank], MPI_DOUBLE, c, counts, displs,
    MPI_DOUBLE, 0, MPI_COMM_WORLD);
```

In order to optimise the speed of the computations on each node, the matrices $B$ and $C$ are stored in aligned memory in a column-major fashion, which simplifies the calls to `MPI_Scatterv` and `MPI_Gatherv`.

### Plots

For the analysis of the matrix multiplication algorithm, the program was run five times for matrix sizes $10\times 10$, $100\times 100$, and $1000\times 1000$ in a single go where after each iteration allocated memory was freed. 
Overall, the program was instructed using 1, 2, 8, 24, 48, and 64 computing nodes in order to analyse the effect of an increasing number processes. The timing was started before initialising any matrix including the given matrices $A$ and $B$ of size $n$ as well as the result matrix.

Although runs with matrix size of less than 1000 benefitted from increasing the number of computing nodes, we focus on the computations done on matrix size $1000 \times 1000$. It should be mentioned that increasing the size to even larger matrices would required the program to handle matrices that were rejected by the memory handler of DelftBlue and therefore would require different approaches. Matrices of row size 1000 require about 8MB of memory and fit easily into RAM.

Figure ([3](#times-of-run-over-the-number-of-processes)) displays the times needed for each iteration. The time required for the computation appears to reduce by increasing the number of computing nodes. The benefit becomes marginal as the number of processes increases beyond 24 across all sizes of the matrix. However, the average times of the matrix multiplication of sizes 100 using 64 nodes turned out to be slower. This is caused an implementation detail: If the number of nodes is larger than half of the number of rows of the matrices, then the last computing node is required to compute many rows. In this case 37 rows which increases the overall running time significantly. The proposed implementation therefore relies on the assumption that twice as many rows than computing nodes exist in order to distribute the work evenly. This assumption is generally applicable to real-world examples where the size of the matrix is much larger than the number of available computing nodes.   

Although not clearly visible in the figure, the first run often took longer that subsequent runs with the same matrix sizes. As the allocated memory of the implemented code is freed after each run we can exclude allocation of memory on the user side from the set of possible explanations if we assume that the operating system has sufficient memory to provide subsequent calls with the same amount of memory. As the program was instructed to require 1GB of RAM and the allocated memory for matrices sums up to around 200MB, this seems to be reasonable. However, the internal buffer size of the MPI operations, in this case Broadcast and Scatter, may require the MPI subprocess to increase the buffer for larger matrices. As the slowdown only happened on the first calls after increasing the matrix sizes this explanation also fits the general experimental setup.
Moreover, some runs of matric sizes 10 and 100 took longer than the job of matrix size 1000. For example, for 24 nodes some two runs were slower. Overall, the trend follows the theoretical idea of reducing the computing time when increasing the number of nodes at least if the matrix size is reasonable large enough. Unnecessarily large computing times for small matrices may be circumvented by falling back to an algorithm that operates only on one node.  



#### Times of run over the number of processes

![FigureOfTimeOverNumProcesses](./figs/MM-product-time-over-procs.svg)

## Exercise 1: Parallel solver for the Poisson equation

### 1.2.2

Running the problem several times with a topology of 4 x 1 yields and optimal value of around 1.93 as suggested by the exercise. For values $\omega \in [1.9,1.99]$ we did a binary-like search in order to find this optimal value. The number of iterations using the optimal value turned out to be $131$ taking around $0.042 [s]$.

precision goal: 0.000100
Number of iterations: 131, omega: 1.93
Delta: 0.000066
(0) Elapsed Wtime       0.041549 s ( 21.5% CPU)
precision goal: 0.000100
Delta: 0.000089
(1) Elapsed Wtime       0.013035 s ( 82.9% CPU)
precision goal: 0.000100
Delta: 0.000099
(2) Elapsed Wtime       0.077678 s ( 14.0% CPU)
precision goal: 0.000100
Delta: 0.000078
(3) Elapsed Wtime       0.057615 s ( 15.5% CPU)

### 1.2.3

